In [1]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import statsmodels.api as sm
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Linear Regression ML Artist Identification Model:

In [2]:
#Authentication - without user
cid = 'e1d252163c854439adf89ee1872895ef'
secret = '196af4bb9d674ad7942fb1c2f57f55fb'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
playlist_link = 'https://open.spotify.com/playlist/3Q2GsAzMUmMcIV2TGNqleu?si=9801a625a0d64160'
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [4]:
# creating dataframe
dataframe = {'Artist' : []}
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #Name
    artist_name = track["track"]["artists"][0]["name"]
    if artist_name == 'Taylor Swift':
        dataframe['Artist'].append(1)
    else:
        dataframe['Artist'].append(0)
        
    for keys, values in sp.audio_features(track["track"]["uri"])[0].items():
        if type(values) == float or type(values) == int:
            
            if keys not in dataframe:
                dataframe[keys] = []
            dataframe[keys].append(values)


In [5]:
df = pd.DataFrame(data=dataframe)
df = df.drop(columns=['mode'])
df

,Artist,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1,0.652,0.802,7,-6.114,0.1810,0.08710,0.000006,0.1480,0.295,170.157,211933,4
1,0,0.654,0.760,0,-3.669,0.0450,0.07970,0.000000,0.2990,0.410,99.945,200787,4
2,0,0.776,0.875,4,-5.535,0.0575,0.03430,0.000000,0.3140,0.849,111.988,211786,4
3,0,0.696,0.893,9,-2.963,0.0760,0.06620,0.000000,0.4730,0.573,97.954,214013,4
4,0,0.645,0.878,5,-3.208,0.1130,0.29400,0.000000,0.0760,0.862,82.324,227027,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0.588,0.749,11,-4.323,0.0580,0.00370,0.000000,0.0813,0.397,130.093,174600,4
96,1,0.359,0.543,7,-7.582,0.0919,0.49200,0.000016,0.1180,0.453,68.534,221307,4
97,1,0.777,0.658,0,-5.191,0.0540,0.07670,0.000000,0.0901,0.633,110.048,190360,4
98,0,0.620,0.805,5,-5.538,0.0417,0.00564,0.000092,0.2850,0.586,122.062,204764,4


In [6]:
# create the actual model
features = df.drop(columns=['Artist'])
labels = df['Artist']

train_vectors, test_vectors, train_labels, test_labels = train_test_split(features, labels, train_size = 0.75, random_state = 314159)

In [7]:
logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
result = logit_model.fit()
print(result.summary() )

         Current function value: 0.352940
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                 Artist   No. Observations:                   75
Model:                          Logit   Df Residuals:                       62
Method:                           MLE   Df Model:                           12
Date:                Sun, 24 Apr 2022   Pseudo R-squ.:                  0.2668
Time:                        19:57:54   Log-Likelihood:                -26.470
converged:                      False   LL-Null:                       -36.102
Covariance Type:            nonrobust   LLR p-value:                   0.08240
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -92.9582   5.28e+04     -0.002      0.999   -1.03e+05    1.03e+05
danceability         4.2344      4.521      0.9

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [8]:
predict = result.predict(sm.add_constant(test_vectors))
final_predict = []
for i in predict:
    if i < 0.5:
        final_predict.append(0)
    else:
        final_predict.append(1)
accuracy_score(test_labels, final_predict) # accuracy score

0.84

# Principal Component Analysis (PCA) Model:

In [9]:
# Normalizing the data before performing PCA
scaler = MinMaxScaler()

features_norm = scaler.fit_transform(features)
features_norm = pd.DataFrame(features_norm)
features_norm

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.626087,0.787078,0.636364,0.511452,0.313287,0.096102,0.000743,0.125126,0.265191,0.761135,0.397105,1.0
1,0.629565,0.725404,0.000000,0.775605,0.036977,0.087873,0.000000,0.315206,0.405795,0.235262,0.307411,1.0
2,0.841739,0.894273,0.363636,0.574006,0.062373,0.037391,0.000000,0.334089,0.942536,0.325462,0.395922,1.0
3,0.702609,0.920705,0.818182,0.851880,0.099959,0.072862,0.000000,0.534240,0.605086,0.220350,0.413843,1.0
4,0.613913,0.898678,0.454545,0.825411,0.175132,0.326163,0.000000,0.034491,0.958430,0.103284,0.518569,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.514783,0.709251,1.000000,0.704948,0.063389,0.003366,0.000000,0.041163,0.389901,0.461064,0.096679,1.0
96,0.116522,0.406755,0.636364,0.352852,0.132263,0.546327,0.001820,0.087362,0.458369,0.000000,0.472539,1.0
97,0.843478,0.575624,0.000000,0.611171,0.055262,0.084538,0.000000,0.052241,0.678445,0.310931,0.223503,1.0
98,0.570435,0.791483,0.454545,0.573682,0.030272,0.005523,0.010541,0.297583,0.620981,0.400914,0.339414,1.0


In [10]:
# creating new training/testing sets based on the newly normalized features
train_vectors_norm, test_vectors_norm, train_labels, test_labels = train_test_split(features_norm, labels, random_state = 314160)

#### Explanation for the next part:
There are 12 features in our model. Danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration (in milliseconds), and time signature. We can do various PCA models selecting only a certain number of features to assess. To find what can be considered a 'good' PCA model, you want the explained variance ratio to be _around_ .8, which will prevent overfitting. Below is the attempt to find the 'best' explained variance ratio, and thus the 'best' PCA model.

In [11]:
pca6 = PCA(n_components = 6)

fit6 = pca6.fit(train_vectors_norm)

pca_train_vectors6 = pca6.transform(train_vectors_norm)
pca_test_vectors6 = pca6.transform(test_vectors_norm)

print(pca6.explained_variance_ratio_)

EVR6 = 0
for val in (pca6.explained_variance_ratio_):
    EVR6 += val
    
print('Total explained variance ratio:', EVR6)

[0.22291939 0.21302126 0.11742368 0.10116306 0.08529594 0.06954937]
Total explained variance ratio: 0.8093727003210414


In [12]:
pca5 = PCA(n_components = 5)

fit5 = pca5.fit(train_vectors_norm)

pca_train_vectors5 = pca5.transform(train_vectors_norm)
pca_test_vectors5 = pca5.transform(test_vectors_norm)

print(pca5.explained_variance_ratio_)

EVR5 = 0
for val in (pca5.explained_variance_ratio_):
    EVR5 += val
    
print('Total explained variance ratio:', EVR5)

[0.22291939 0.21302126 0.11742368 0.10116306 0.08529594]
Total explained variance ratio: 0.739823328722721


#### Conclusion:
I started off by creating a PCA with 6 features, which has an explained variance ratio of 0.809, or 0.81. To try to see if I could get the model to be even closer to .80, I went one lower (going higher would increase the EVR) and created a PCA with 5 features, which resulted in an EVR of 0.739, or 0.74. Since 0.81 is closer to 0.8 than 0.74, the PCA I will be using moving forward is the PCA with 6 features.

In [13]:
X_pca = pca5.fit_transform(features_norm)
df_pca = pd.DataFrame(X_pca)
df_pca

,0,1,2,3,4
0,0.001506,0.272877,0.222945,0.224706,0.010536
1,-0.119090,-0.413063,0.108108,-0.121828,-0.030319
2,-0.375628,-0.222911,-0.316640,0.130373,0.066571
3,-0.422862,0.321119,-0.250100,-0.054644,0.153047
4,-0.273816,-0.081902,-0.190240,0.014037,-0.026225
...,...,...,...,...,...
95,-0.106590,0.555763,0.032535,0.023946,-0.319045
96,0.571665,0.220573,0.035815,-0.206735,0.088771
97,-0.037570,-0.531377,-0.124120,0.030487,-0.230963
98,-0.194196,-0.021519,-0.015061,0.040279,0.026852


#### Creating a new linear regression model based on the PCA results:

In [14]:
train_vectors_pca, test_vectors_pca, train_labels, test_labels = train_test_split(df_pca, labels, random_state = 314170)

In [15]:
logit_model_pca = sm.Logit(train_labels, sm.add_constant(train_vectors_pca))
result_pca = logit_model_pca.fit()
print(result_pca.summary())

Optimization terminated successfully.
         Current function value: 0.334448
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 Artist   No. Observations:                   75
Model:                          Logit   Df Residuals:                       69
Method:                           MLE   Df Model:                            5
Date:                Sun, 24 Apr 2022   Pseudo R-squ.:                 0.08851
Time:                        19:58:11   Log-Likelihood:                -25.084
converged:                       True   LL-Null:                       -27.519
Covariance Type:            nonrobust   LLR p-value:                    0.4318
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1934      0.414     -5.302      0.000      -3.004      -1.383
0              1.2621      1.

In [16]:
predict_pca = result_pca.predict(sm.add_constant(test_vectors_pca))
final_predict_pca = []
for i in predict_pca:
    if i < 0.5:
        final_predict_pca.append(0)
    else:
        final_predict_pca.append(1)
accuracy_score(test_labels, final_predict_pca) # accuracy score for the PCA model

0.64

### PCA Model Conclusion:
The accuracy score for the original logistic regression model was 0.84. The accuracy score for the PCA model was 0.64. Since the accuracy score decreased significantly after applying PCA, it is clear that PCA made the logistic regression model worse. Therefore, PCA is not a good choice for this particular scenario.

# SVC

In [17]:
# create the actual model
features = df.drop(columns=['Artist'])
labels = df['Artist']

train_vectors, test_vectors, train_labels, test_labels = train_test_split(features, labels, train_size = 0.75, random_state = 314159)

In [18]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1e-3, 0.01,0.1,1,10,100], 'gamma':[1e-6,1e-5,1e-4,1e-3,0.01,0.1]}
my_model = SVC()
clf = GridSearchCV(my_model, parameters)
clf.fit(train_vectors, train_labels)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1],
                         'kernel': ('linear', 'rbf')})

In [19]:
pca6

PCA(n_components=6)

In [20]:
pca_train = pca6.fit_transform(train_vectors)
my_model = SVC(kernel = clf.get_params()['estimator__kernel'], C = clf.get_params()['estimator__C'], gamma = clf.get_params()['estimator__gamma'])
results = my_model.fit(pca_train, train_labels)

In [21]:
pca_test = pca6.fit_transform(test_vectors)
predict_test = results.predict(pca_test)

accuracy_score(test_labels, predict_test)

0.8

# Perceptron Model

In [25]:
from sklearn.linear_model import Perceptron
model = Perceptron()
model.fit(train_vectors, train_labels)
y_pred = model.predict(test_vectors)

print(classification_report(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred, labels=labels))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91        21
           1       0.00      0.00      0.00         4

    accuracy                           0.84        25
   macro avg       0.42      0.50      0.46        25
weighted avg       0.71      0.84      0.77        25

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0 21  0]
 [ 0  0  0 ...  0  4  0]]


/home/wehnerjo/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wehnerjo/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/wehnerjo/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
print("Accuracy:", model.score(train_vectors,train_labels))

Accuracy: 0.8133333333333334


### Perceptron Model Conclusion:
The accuracy score from the Perceptron model is approximately 0.8133. Although the model does a pretty good job of classifying the artist into the appropriate classification, the original logistic regression model is the best model. As the original logistic regression model had an accuracy of 0.84, the Perceptron model would not be the best for this scenario.
